<a href="https://colab.research.google.com/github/roy-sib2002/github-pages/blob/main/imd_data_t_min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import glob
import re

# Grid dimensions and coordinates (same as Tmax)
nlat, nlon = 31, 31
lats = np.linspace(7.5, 37.5, nlat)
lons = np.linspace(67.5, 97.5, nlon)

# Directories for input and output
data_dir = '/content/drive/MyDrive/Colab Notebooks/tmin_data'
output_dir = '/content/drive/MyDrive/Colab Notebooks/t_min_output'

# Create output folder if not exists
os.makedirs(output_dir, exist_ok=True)

# Get all GRD files
file_paths = sorted(glob.glob(os.path.join(data_dir, "Mintemp_MinT_*.GRD")))

for file_path in file_paths:
    filename = os.path.basename(file_path)

    # Extract year
    match = re.search(r'(\d{4})', filename)
    if not match:
        print(f"Year not found in file name: {filename}")
        continue

    year = int(match.group(1))
    ndays = 366 if pd.Timestamp(f"{year}-12-31").is_leap_year else 365

    print(f"Reading Tmin file: {filename} ({ndays} days)")

    raw = np.fromfile(file_path, dtype=np.float32)
    expected_shape = (ndays, nlat, nlon)
    expected_elements = ndays * nlat * nlon

    if raw.size != expected_elements:
        print(f"Reshape failed for {filename}. Raw size = {raw.size}, expected = {expected_elements}. Skipping.")
        continue

    try:
        data = raw.reshape(expected_shape)
    except ValueError:
        print(f" Reshape error in {filename}. Skipping.")
        continue

    dates = pd.date_range(start=f"{year}-01-01", periods=ndays)

    ds = xr.Dataset(
        {
            'tmin': (['time', 'lat', 'lon'], data)
        },
        coords={
            'time': dates,
            'lat': lats,
            'lon': lons
        },
        attrs={
            'title': f'Daily Tmin from IMD file ({year})',
            'description': 'Converted from .GRD to NetCDF',
            'source': 'IMD',
            'resolution': '1.0 x 1.0 degree'
        }
    )

    # Save as NetCDF
    output_path = os.path.join(output_dir, f"tmin_IMD_{year}.nc")
    ds.to_netcdf(output_path)
    print(f"Saved: {output_path}")

print("All Tmin NetCDF files saved to 't_min_output'")


📂 Reading Tmin file: Mintemp_MinT_1951.GRD (365 days)
✅ Saved: /content/drive/MyDrive/Colab Notebooks/t_min_output/tmin_IMD_1951.nc
📂 Reading Tmin file: Mintemp_MinT_1952.GRD (366 days)
✅ Saved: /content/drive/MyDrive/Colab Notebooks/t_min_output/tmin_IMD_1952.nc
📂 Reading Tmin file: Mintemp_MinT_1953.GRD (365 days)
✅ Saved: /content/drive/MyDrive/Colab Notebooks/t_min_output/tmin_IMD_1953.nc
📂 Reading Tmin file: Mintemp_MinT_1954.GRD (365 days)
✅ Saved: /content/drive/MyDrive/Colab Notebooks/t_min_output/tmin_IMD_1954.nc
📂 Reading Tmin file: Mintemp_MinT_1955.GRD (365 days)
✅ Saved: /content/drive/MyDrive/Colab Notebooks/t_min_output/tmin_IMD_1955.nc
📂 Reading Tmin file: Mintemp_MinT_1956.GRD (366 days)
✅ Saved: /content/drive/MyDrive/Colab Notebooks/t_min_output/tmin_IMD_1956.nc
📂 Reading Tmin file: Mintemp_MinT_1957.GRD (365 days)
✅ Saved: /content/drive/MyDrive/Colab Notebooks/t_min_output/tmin_IMD_1957.nc
📂 Reading Tmin file: Mintemp_MinT_1958.GRD (365 days)
✅ Saved: /content/driv